# TM10007 Assignment template

In [17]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [3]:
# Data loading functions. Uncomment the one you want to use
#from worcgist.load_data import load_data
#from worclipo.load_data import load_data
#from worcliver.load_data import load_data
#from ecg.load_data import load_data

import pandas as pd 
import os 

def load_data():
    this_directory = os.getcwd()
    data = pd.read_csv(os.path.join(this_directory, 'Lipo_radiomicFeatures.csv'), index_col=0)
    return data

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')



The number of samples: 115
The number of columns: 494


In [4]:
import pandas as pd

# Load the dataset
file_path = "data.csv"  # Ensure the dataset is in the same directory as the notebook
df = pd.read_csv(file_path)

# Display basic info and first few rows
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Columns: 495 entries, ID to PREDICT_original_phasef_phasesym_entropy_WL3_N5
dtypes: float64(468), int64(25), object(2)
memory usage: 444.9+ KB


,ID,label,PREDICT_original_sf_compactness_avg_2.5D,PREDICT_original_sf_compactness_std_2.5D,PREDICT_original_sf_rad_dist_avg_2.5D,PREDICT_original_sf_rad_dist_std_2.5D,PREDICT_original_sf_roughness_avg_2.5D,PREDICT_original_sf_roughness_std_2.5D,PREDICT_original_sf_convexity_avg_2.5D,PREDICT_original_sf_convexity_std_2.5D,...,PREDICT_original_phasef_phasesym_median_WL3_N5,PREDICT_original_phasef_phasesym_std_WL3_N5,PREDICT_original_phasef_phasesym_skewness_WL3_N5,PREDICT_original_phasef_phasesym_kurtosis_WL3_N5,PREDICT_original_phasef_phasesym_peak_WL3_N5,PREDICT_original_phasef_phasesym_peak_position_WL3_N5,PREDICT_original_phasef_phasesym_range_WL3_N5,PREDICT_original_phasef_phasesym_energy_WL3_N5,PREDICT_original_phasef_phasesym_quartile_range_WL3_N5,PREDICT_original_phasef_phasesym_entropy_WL3_N5
0,Lipo-001_0,liposarcoma,0.368434,0.262371,26.200395,8.174390,20.940000,13.551937,0.694139,0.227848,...,0.049726,0.196019,0.927821,-0.407270,0.0,0,0.625304,3238.304991,0.315148,14.462361
1,Lipo-002_0,liposarcoma,0.660571,0.179013,55.155046,9.318741,9.546044,7.051937,0.871182,0.125279,...,0.036186,0.167471,1.137972,0.411968,0.0,0,0.563659,15364.724390,0.251255,17.170274
2,Lipo-003_0,lipoma,0.365191,0.157712,30.577026,8.963647,17.695139,10.772365,0.704710,0.137529,...,0.128167,0.220546,0.829650,-0.520344,0.0,0,0.705679,3771.798158,0.369366,14.394812
3,Lipo-004_0,liposarcoma,0.372210,0.169761,94.006805,33.512409,21.049148,15.928362,0.754497,0.186559,...,0.000000,0.107987,3.130011,10.399652,0.0,0,0.438399,4707.658397,0.000000,15.733482
4,Lipo-005_0,lipoma,0.369120,0.167485,94.612701,31.250821,17.059573,15.239905,0.774545,0.192922,...,0.000000,0.087662,3.284943,11.840826,0.0,0,0.356330,6171.440113,0.000000,16.682154


In [5]:
# Check for missing values
missing_values = df.isnull().sum().sum()
print(f"Total missing values: {missing_values}")

# Encode the target variable
df['label'] = df['label'].map({'lipoma': 0, 'liposarcoma': 1})

# Drop ID column as it is not useful for training
df.drop(columns=['ID'], inplace=True)

# Normalize numerical features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df.iloc[:, 1:] = scaler.fit_transform(df.iloc[:, 1:])

# Display dataset after preprocessing
df.head()

Total missing values: 0


/var/folders/7c/k7s45g950s74y_jl8zgdy5hh0000gn/T/ipykernel_92442/3430236790.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.65205945  0.31068168  1.16645157 -1.61480058  0.73856663 -0.86600192
  0.9525091   0.09673921 -0.75903069 -0.01023203  0.41765292  1.05948034
  1.16645157  0.84553786  0.84553786 -1.40085811 -0.54508821 -0.01023203
  0.73856663 -0.43811698  0.9525091   0.63159539  0.9525091   0.52462415
  0.63159539  0.63159539 -0.2241745   0.52462415 -0.33114574 -1.93571429
  1.05948034 -0.65205945  0.52462415 -2.04268553 -2.68451295 -0.33114574
  0.84553786  0.73856663  0.09673921  0.73856663 -0.75903069  0.20371044
 -2.256628   -0.01023203  0.84553786  0.20371044  0.73856663 -1.40085811
  0.9525091  -1.72177182  0.20371044 -0.01023203 -0.43811698  0.31068168
 -1.50782934  0.41765292 -1.50782934  0.52462415 -1.72177182  0.20371044
  0.73856663  1.59433652  1.05948034  0.09673921 -1.0799444   1.16645

,label,PREDICT_original_sf_compactness_avg_2.5D,PREDICT_original_sf_compactness_std_2.5D,PREDICT_original_sf_rad_dist_avg_2.5D,PREDICT_original_sf_rad_dist_std_2.5D,PREDICT_original_sf_roughness_avg_2.5D,PREDICT_original_sf_roughness_std_2.5D,PREDICT_original_sf_convexity_avg_2.5D,PREDICT_original_sf_convexity_std_2.5D,PREDICT_original_sf_cvar_avg_2.5D,...,PREDICT_original_phasef_phasesym_median_WL3_N5,PREDICT_original_phasef_phasesym_std_WL3_N5,PREDICT_original_phasef_phasesym_skewness_WL3_N5,PREDICT_original_phasef_phasesym_kurtosis_WL3_N5,PREDICT_original_phasef_phasesym_peak_WL3_N5,PREDICT_original_phasef_phasesym_peak_position_WL3_N5,PREDICT_original_phasef_phasesym_range_WL3_N5,PREDICT_original_phasef_phasesym_energy_WL3_N5,PREDICT_original_phasef_phasesym_quartile_range_WL3_N5,PREDICT_original_phasef_phasesym_entropy_WL3_N5
0,1,-0.636358,1.178671,-0.892099,-0.591539,0.105944,-0.163109,-0.525603,0.864401,0.279704,...,0.473530,0.960241,-0.912616,-0.738021,0.0,0,0.776568,-0.388714,1.048810,-0.322777
1,1,0.868499,-0.001097,0.324289,-0.445390,-0.781146,-0.571225,0.569888,-0.328527,-1.144128,...,0.194855,0.283867,-0.648104,-0.546022,0.0,0,0.238135,1.207257,0.517048,1.221523
2,0,-0.653059,-0.302556,-0.708236,-0.490740,-0.146689,-0.337630,-0.460197,-0.186049,0.100405,...,2.087912,1.541353,-1.036182,-0.764522,0.0,0,1.478594,-0.318501,1.500051,-0.361300
3,1,-0.616905,-0.132029,1.956455,2.644459,0.114441,-0.013900,-0.152128,0.384195,0.873545,...,-0.549889,-1.125449,1.859232,1.794725,0.0,0,-0.855941,-0.195331,-1.574077,0.402132
4,0,-0.632823,-0.164240,1.981909,2.355624,-0.196172,-0.057127,-0.028077,0.458204,0.778158,...,-0.549889,-1.606982,2.054241,2.132483,0.0,0,-1.572767,-0.002681,-1.574077,0.943152
